In [55]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_KEY')
MODEL = input("Enter your model name: ")

In [56]:
# from langchain_openai.chat_models import ChatOpenAI
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)
# def embed(text):
#     return embeddings.embed_query(text)

In [57]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
chain = model | parser 
# chain.invoke("Tell me a joke")

#### Load our pdf 

In [58]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("100-Machine-Learning-Interview-Questions-and-Answers.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='100 Machine Learning Interview Questions and\nAnswers\n1. Please Explain Machine Learning, Artificial Intelligence, And Deep Learning?\nMachine learning is defined as a subset of Artificial Intelligence, and it contains the techniques\nwhich enable computers to sort things out from the data and deliver Artificial Intelligence\napplications. Artificial Intelligence (AI) is a branch of computer science that is mainly focused on\nbuilding smart machines that can perform certain tasks that mainly require human intelligence. It\nis the venture to replicate or simulate human intelligence in machines.\nDeep learning can be defined as a class of machine learning algorithms in Artificial Intelligence\nthat mainly uses multiple layers to cumulatively extract higher-level features from the given raw\ninput.\n2. How Difficult Is Machine Learning?\nMachine Learning is huge and comprises a lot of things. Therefore, it will take more than six\nmonths to learn Machine Learning 

In [59]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer this question, reply "I Don't Know".

Context:{context}

Question:{question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context",question="Here is a question"))


Answer the question based on the context below. If you can't 
answer this question, reply "I Don't Know".

Context:Here is some context

Question:Here is a question



In [60]:
chain = prompt | model | parser

In [61]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}}}

In [62]:
chain.invoke(
    {
        "context": "The name I was given was Ajay ",
        "question": "What is my name?"
    }
)

'Your name is Ajay.'

In [63]:
### Doc Array in memory search
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages,
    embedding=embeddings
)

In [64]:
retriever = vectorstore.as_retriever()

In [65]:
retriever.invoke("Projects")

[Document(page_content='A/B testing is defined as a basic randomized control experiment. It is used to compare two\nversions of a variable to find out which one among them performs better in a controlled\nenvironment.\nA/B Testing can be best used to compare two models to check which one is the\nbest-recommended product to a customer.\n45. Explain Marginalisation And Its Process?\nMarginalization is a method that requires the summing of the possible values of one variable to\ndetermine the marginal contribution of another variable.\nP(X=x) = ∑YP(X=x,Y)\n46. What Is Cluster Sampling?\nCluster sampling is defined as a type of sampling method. With cluster sampling, the\nresearchers usually divide the population into separate groups or sets, known as clusters. Then,\na random sample of clusters is picked from the population. Then the researcher conducts their\nanalysis on the data from the collected sampled clusters.\n47. Explain The Term“Curse Of Dimensionality”?\nThe curse of dimensiona

In [67]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter('question') | retriever,
        "question": itemgetter('question')
    }
    | prompt
    | model
    | parser
)

# chain.invoke({"question":"Projetcs?"})

In [68]:
itemgetter("question")({"question":"projects?"})

'projects?'

In [69]:
questions = ["Can You Name The Different Machine Learning Algorithms?",
             "What Is Overfitting?"]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()


Question: Can You Name The Different Machine Learning Algorithms?
Answer: I Don't Know.

Question: What Is Overfitting?
Answer: Answer based on provided context:

Overfitting is a type of modeling error that results in the failure to predict or guess the future observations effectively or fit additional data in the model that already exists.

